In [ ]:
import pandas as pd

df = pd.read_parquet('Data/yellow_tripdata_2023-01.parquet', engine='pyarrow')



In [ ]:
before = len(df)
df = df.drop_duplicates()

after  = len(df)
removed = before - after
print(f"Duplicated file with number of {removed} was deleted")


In [ ]:
df = df[df['passenger_count'] != 0]
df = df[df['trip_distance'] != 0]

print(f"Data was clenead from rows where passenger_count = 0 and trip_distance = 0")


In [ ]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
print(f"Tpep_pickup_datetime was successfully change to datetime")


In [ ]:
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
print(f"Tpep_dropoff_datetime was successfully change to datetime")

In [ ]:
from sqlalchemy import create_engine
from io import StringIO
from dotenv import load_dotenv
import os

load_dotenv()

user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")


engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{db_name}")

empty_df = df.iloc[0:0]
empty_df.to_sql("trips_cleaned", engine, if_exists="replace", index=False)
print(f"Create empty table")

conn = engine.raw_connection()
cursor = conn.cursor()

buffer = StringIO()
df.to_csv(buffer,index=False,header=False)
buffer.seek(0)

cursor.copy_expert('COPY trips_cleaned FROM STDIN WITH CSV', buffer)

conn.commit()
print(f"Data was successfully added to database")

cursor.close()